In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [58]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(         # input =28x28x1 , RF: 3x3
        nn.Conv2d(1, 16, 3) ,     ## out=28x28x16
         nn.ReLU(),          
         nn.BatchNorm2d(16),  
         nn.Dropout2d(0.1),
         nn.Conv2d(16, 16, 3),    ## out=28x28x32,input=28x28x16,RF=5x5
         nn.ReLU(),          
         nn.BatchNorm2d(16),  
         nn.Dropout2d(0.1),     
         nn.Conv2d(16, 32, 3),    ## out=28x28x32,input=28x28x16,RF=5x5
                 nn.ReLU(),          
         nn.BatchNorm2d(32),  
         nn.Dropout2d(0.1)      
        )

        self.transition_layer1= nn.Sequential( 
            nn.Conv2d(32,16,1),              ## out= 14x14x16, input=14x14x32
            nn.ReLU(),                         # input =28x28x32
            nn.MaxPool2d(2, 2),               ## out= 14x14x32

            )      

        self.conv2 = nn.Sequential(       # input=14x14x16
        nn.Conv2d(16, 16, 3),              ## out=12 x12 x 32
                         nn.ReLU(),          
         nn.BatchNorm2d(16),  
         nn.Dropout2d(0.1) , 
        nn.Conv2d(16, 16, 3),             ## out= 10x10x16, input= 12x12x32
                         nn.ReLU(),          
         nn.BatchNorm2d(16),  
         nn.Dropout2d(0.1) , 
          nn.Conv2d(16, 32, 3,padding=1),             ## out= 10x10x16, input= 12x12x32
                         nn.ReLU(),          
         nn.BatchNorm2d(32),  
         nn.Dropout2d(0.1)  
          )

       
        self.conv_final=nn.Conv2d(32, 10, 1, bias=False) #input-5x5x16 Output: 5x5x32 
        self.gap = nn.AvgPool2d(5)  
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.transition_layer1(x)
        x =self.conv2(x)
        
        x = self.conv_final(x)
        x=self.gap(x)
        x=x.view(-1,10)  
        return F.log_softmax(x)

In [59]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
         Dropout2d-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
             ReLU-10           [-1, 32, 22, 22]               0
      BatchNorm2d-11           [-1, 32, 22, 22]              64
        Dropout2d-12           [-1, 32, 22, 22]               0
           Conv2d-13           [-1, 16, 22, 22]             528
             ReLU-14           [-1, 16,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [60]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [61]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [64]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

for epoch in range(1, 20):
    print('\nEpoch {} : '.format(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch 1 : 


loss=0.05222932621836662 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.47it/s]



Test set: Average loss: 0.0532, Accuracy: 9834/10000 (98.34%)


Epoch 2 : 


loss=0.038857780396938324 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.58it/s]



Test set: Average loss: 0.0395, Accuracy: 9882/10000 (98.82%)


Epoch 3 : 


...


Test set: Average loss: 0.0175, Accuracy: 9944/10000 (99.44%)


Epoch 18 : 


loss=0.03112637810409069 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.98it/s]



Test set: Average loss: 0.0186, Accuracy: 9945/10000 (99.45%)


Epoch 19 : 


loss=0.012066666036844254 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.00it/s]



Test set: Average loss: 0.0191, Accuracy: 9947/10000 (99.47%)

